In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [30]:
factor_returns = pd.read_csv("../Analysis/factors_returns_multi.csv")
style_factors = list(factor_returns.columns)[1:11]
style_factor_returns = factor_returns[["Date"]+style_factors]
factors = pd.read_csv("../Factors/Factors.csv")

def strategy(date):
    predicted_factor_return = style_factor_returns.loc[style_factor_returns["Date"]==date]
    predicted_factor_return = predicted_factor_return[style_factors].values[0]
    predicted_factor_return = predicted_factor_return.sort_values(ascending=False)
    top = predicted_factor_return.index[0]
    return top
    

In [29]:
predicted_factor_return = style_factor_returns.loc[:, "2021-01-04"]

KeyError: '2021-01-04'

In [41]:
predicted_factor_return = style_factor_returns.loc[style_factor_returns["Date"]=="2021-01-04"]
predicted_factor_return = predicted_factor_return.set_index("Date")
# predicted_factor_return = predicted_factor_return[style_factors].values[0]
predicted_factor_return = predicted_factor_return.sort_values(by="2021-01-04", ascending=False)
# predicted_factor_return.sort_values(by="2021-01-04", ascending=False)
top = predicted_factor_return.index[0]
print(top)

KeyError: '2021-01-04'

In [9]:
predicted_factor_return
predicted_factor_return = predicted_factor_return.sort_values(ascending=False)
top_5 = predicted_factor_return.index[:5]

In [14]:
factor_returns.columns

Index(['Date', 'Momentum_1', 'Momentum_2', 'Momentum_3', 'Turnover',
       'Volitility_1', 'Volitility_2', 'Volitility_3', 'Financial Quality',
       'EP', 'BP', 'industry_Basic Materials',
       'industry_Communication Services', 'industry_Consumer Cyclical',
       'industry_Consumer Defensive', 'industry_Energy',
       'industry_Financial Services', 'industry_Healthcare',
       'industry_Industrials', 'industry_Real Estate', 'industry_Technology',
       'industry_Utilities'],
      dtype='object')

In [12]:
factor_returns = pd.read_csv("../Analysis/factors_returns_multi.csv")

In [16]:
list(factor_returns.columns)[1:11]

['Momentum_1',
 'Momentum_2',
 'Momentum_3',
 'Turnover',
 'Volitility_1',
 'Volitility_2',
 'Volitility_3',
 'Financial Quality',
 'EP',
 'BP']